# Flyer Basics

We want to understand how to build a *Flyer* in "BlueSky" to support various types of fly scans and remote data loggers.  The data about Flyers is spread about the standard documentation.  We need some clarity and a few examples that build complexity incrementally.

The basic notion of a Flyer is that it directs an *external controller* (we'll call it the *controller* here) to perform some data collection process.  Usually, a *controller* is used to collect data at rates beyond the capabilities of BlueSky plans and the RunEngine.  Examples could be waveforms from a storage oscilloscope or a continuous motion scan of a diffractometer.

This notebook will show the basic requirements for a Flyer and build a simple working example you can use a template for your own work.  (Our inspiration to create this [basic flyer tutorial](https://github.com/prjemian/ipython_mintvm/issues/3#issuecomment-385058577) was the goal of operating various fly scans at the APS, such as the [USAXS fly scan](https://github.com/APS-USAXS/ipython-usaxs/issues/3), from BlueSky.  The examples we found before we started this project quickly became too instrument-specific to serve as tutorials.)


## Python imports and definitions 

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd, BluSky, and databroker packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the BlueSky RunEngine.
* Create an instance of the databroker using [`databroker.temp_config()`](https://nsls-ii.github.io/databroker/generated/databroker.temp_config.html), establishing a temporary, disposable yet fully-functional databroker for just this tutorial session.
* Subscribe that databroker instance to the RunEngine so it receives the document stream each time we run a plan.


In [1]:
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.from_config(databroker.temp_config())
RE.subscribe(db.insert)

0

### Bare Minimum Requirements for a Flyer <a class="anchor" id="flyer-requirements" />

In BlueSky, a [Flyer](http://nsls-ii.github.io/bluesky/async.html?highlight=flyer#flying) is an [`ophyd.Device`](http://nsls-ii.github.io/ophyd/device-overview.html) that meets the [Fly-able interface](http://nsls-ii.github.io/ophyd/architecture.html#fly-able-interface), which has three methods:

1. Kickoff - begin accumulating data
1. Complete - BlueSky tells the Flyer that BlueSky is ready to receive data
1. Collect - the device provides the data to BlueSky

The first two methods [must return](http://nsls-ii.github.io/bluesky/hardware.html#kickoff) an instance of [`ophyd.status.Status`](http://nsls-ii.github.io/ophyd/generated/ophyd.status.Status.html) (a.k.a. a *status* object).  For our code, we choose a specialized version of `Status()`, the [`DeviceStatus()`](http://nsls-ii.github.io/ophyd/generated/ophyd.status.DeviceStatus.html) object, based on the [explanation provided in discussion](https://github.com/NSLS-II/tutorial-docker/pull/7#discussion_r185054364).

The `collect()` method requires a companion `describe_collect()` that informs the RunEngine what kind of data to expect from `collect()`.  If no timestamp information is provided from the *controller*, then do as we show here and use the workstation's clock to provide a timestamp for the event.

This example (which does absolutely nothing) meets the bare minimum requirement.

In [2]:
class BareMinimumFlyer(ophyd.Device):

    def kickoff(self):
        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        complete_status = ophyd.DeviceStatus(self)
        complete_status._finished(success=True)
        return complete_status

    def collect(self):
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        return {self.name: {}}


flyer = BareMinimumFlyer(name="flyer")
print(flyer.complete())
print(list(flyer.collect()))

# if this next step succeeds, it's proof that we did this right!
RE(bluesky.plans.fly([flyer]))

DeviceStatus(device=flyer, done=True, success=True)
[{'data': {}, 'timestamps': {}, 'time': 1525113618.4073882}]


('a2fab66f-e0f9-443d-b85b-867f5c9d65cb',)

Only two `documents` were emitted from the `RunEngine`: `start` and `stop` since we generated no data content.

In [3]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 1,
   'time': 1525113618.409176,
   'uid': 'a2fab66f-e0f9-443d-b85b-867f5c9d65cb'}),
 ('stop',
  {'exit_status': 'success',
   'num_events': {'flyer': 1},
   'run_start': 'a2fab66f-e0f9-443d-b85b-867f5c9d65cb',
   'time': 1525113618.4396124,
   'uid': 'fdd375b4-e9d6-4bd2-8f00-f0aae26c2225'})]

### Flyer : a starting template <a class="anchor" id="flyer-template" />

The `BareMinimumFlyer` is a good start to use a Flyer but we'll need to add a few more things to make a good template.  The first thing to do is to make the status object known to any method of the class.  We'll call it `self.complete_status` and it will tell us if the *controller* is finished.  In the constructor (`__init__()`), we set it to `None`, the value we expect when not *flying*.  Since we **need** a constructor, we must remember to call the constructor of the superclass as well or our `ophyd.Device` will not work correctly.

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None

Our *controller* could take some time to signal that it is finished (seconds to minutes, at least).  We need a way to detect this completion.  We can do that either by polling the PV or by subscribing to a callback on the completion event.  We'll make that choice when we implement the actual activity.  Here, intend to wait in a polling loop.  Since the polling loop is an activity that does not return until the *controller* is done, we must do that waiting in a thread (consider an [alternative suggestion to use `epics.ca.CAThread`](https://github.com/NSLS-II/tutorial-docker/pull/7#discussion_r184905960)) separate from that of the RunEngine.  (We do not want to block the RunEngine thread so it is free to respond to other activities, such as data from other streams or the user inerface.)  So, we run `my_activity()` in a separate thread that is called from `kickoff()`:

        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

The basic outline of `my_activity()` is:

    def my_activity(self):
        # tell the *controller* to do its work
        # wait for for it to tell us it is done
        self.complete_status._finished(success=True)

The waiting step will *block the thread* in which `my_activity()` is running but that's OK since this is separate from the RunEngine's thread.

Within `my_activity()`,  we've left two comments starting with `# TODO: ` marking where we need to add code that will complete the specifics of our final project.  Since this tutorial develops a starting template for that project (and others), we leave these comments as-is.

We've also added some diagnostic reporting (calls to `logger.info(...)`) to build out the next example:

In [4]:
class MyFlyer(ophyd.Device):
    """
    starting template for a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self.complete_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self.complete_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._comcomplete_statuspletion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        kickoff_status = ophyd.DeviceStatus(self)
        self.complete_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self.complete_status is None:
            raise RuntimeError("No collection in progress")

        return self.complete_status

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self.complete_status = None
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {self.name: {}}


In [5]:
ifly = MyFlyer(name="ifly")

### Diagnostics  <a class="anchor" id="Diagnostics" />

When building a `Flyer`, it is useful to have some diagnostics in place.  Already, we have been using some of these, including printing interim messages via calls to `logger.info(...)`.  Another useful diagnostic step is to call each of the methods individually to make sure they are acting as expected.

1. create an instance of the `Flyer`

    flyer = MyFlyer(name="flyer")

1. verify that `kickoff()` returns a status that is "Done"

    status = flyer.kickoff()
    status.done

1. verify that `complete()` returns a status that is "Done"

    status = flyer.complete()
    status.done

1. verify that `describe_collect()` returns a dictionary

    d = flyer.describe_collect()
    d

1. verify that `collect()` returns a generator

    g = flyer.collect()
    g

1. verify that generator is a list of data dictionaries

    list(g)


Apply some of those steps here (we'll skip testing the `ifly.complete()` method when not flying since it raises a `RuntimeError` exception if data collection is not in progress):

In [6]:
ifly.describe_collect()

{'ifly': {}}

In [7]:
list(ifly.collect())

[{'data': {}, 'time': 1525113618.6107328, 'timestamps': {}}]

Now, run this fly scan:

In [8]:
RE(bluesky.plans.fly([ifly]))

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/mintadmin/Apps/BlueSky/lib/python3.6/site-packages/ophyd/device.py", line 892, in __getattr__
    cpt = self._sig_attrs[name]
KeyError: '_comcomplete_statuspletion_status'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mintadmin/Apps/BlueSky/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/mintadmin/Apps/BlueSky/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-31daf106af24>", line 27, in my_activity
    logger.info("activity() complete. status = " + str(self._comcomplete_statuspletion_status))
  File "/home/mintadmin/Apps/BlueSky/lib/python3.6/site-packages/ophyd/device.py", line 895, in __getattr__
    raise AttributeError(name)
AttributeError: _comcomplete_statuspletion_status



('39a3757a-ce1e-473d-8887-1ed1cf3e4409',)

In [9]:
db[-1].stream_names

['ifly']

In [10]:
db[-1].table("ifly")

""
seq_num


Still only two `documents` were emitted from the `RunEngine`

In [11]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 2,
   'time': 1525113618.6357167,
   'uid': '39a3757a-ce1e-473d-8887-1ed1cf3e4409'}),
 ('stop',
  {'exit_status': 'success',
   'num_events': {'ifly': 1},
   'run_start': '39a3757a-ce1e-473d-8887-1ed1cf3e4409',
   'time': 1525113618.6681423,
   'uid': '8bbb1906-0253-4cfa-a729-d75147c71fcc'})]

## First working Flyer - trivial data <a class="anchor" id="trivial-data-flyer" />

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/062d1765023a4d9...388eb30304e51).

To collect data, we need to modify both the `collect()` *and* the `describe_collect()` methods.  BlueSky needs to know what kind of data to expect from this Flyer, so that it can generate the correct `descriptor` document.

For the *most* trivial case, we'll return a single number (`1.2345`) as the result of the first working Flyer.

In the `describe_collect()` method, we create a dictionary that describes the data to be collected:

        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

Then, in the `collect()` method, add the actual data collection code:

        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d


In [12]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self.complete_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self.complete_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self.complete_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        kickoff_status = ophyd.DeviceStatus(self)
        self.complete_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self.complete_status is None:
            raise RuntimeError("No collection in progress")

        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self.complete_status = None
        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d

As before, create a new instance of the *revised* `MyFlyer` class.

In [13]:
ifly = MyFlyer(name="ifly")

In [14]:
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

output from describe_collect() :  {'ifly': {'x': {'source': 'fictional', 'dtype': 'number', 'shape': []}}}
list output from collect() :  [{'time': 1525113618.9005554, 'data': {'x': 1.2345}, 'timestamps': {'x': 1525113618.9005554}}]


Running this flyer with the RunEngine seems anticlimactic but the lack of exceptions tells us that it ran and we get a UUID at the end.

In [15]:
RE(bluesky.plans.fly([ifly]))

('41aa0130-ce4b-4eed-b836-2860f8758ebb',)

We next query the last scan in the databroker and show a table of the stream from `collect()`:

In [16]:
h = db[-1]
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2018-04-30 14:40:18.931159,1.2345


Now, there are two more document types emitted: `descriptor` and `event`.  Only one `event` since there is only one data point.

In [17]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 3,
   'time': 1525113618.9171739,
   'uid': '41aa0130-ce4b-4eed-b836-2860f8758ebb'}),
 ('descriptor',
  {'data_keys': {'x': {'dtype': 'number', 'shape': [], 'source': 'fictional'}},
   'hints': {},
   'name': 'ifly',
   'object_keys': {'ifly': ['x']},
   'run_start': '41aa0130-ce4b-4eed-b836-2860f8758ebb',
   'time': 1525113618.9233115,
   'uid': '4642f22c-a4aa-4e95-811e-dd7ce7858e3e'}),
 ('event',
  {'data': {'x': 1.2345},
   'descriptor': '4642f22c-a4aa-4e95-811e-dd7ce7858e3e',
   'filled': {},
   'seq_num': 1,
   'time': 1525113618.9311585,
   'timestamps': {'x': 1525113618.9311585},
   'uid': '4850706f-b91f-4f4b-a303-68f7169579a0'}),
 ('stop',
  {'exit_status': 'success',
   'num_events': {'ifly': 1},
   'run_start': '41aa0130-ce4b-4eed-b836-2860f8758ebb',
   'time': 1525113618.942718,
   'uid': '3db24931-a8fd-4468-b232-94821607f520'})]

## Flyer that "collects" 1-D array data  <a class="anchor" id="simple-1d-array-flyer" />

We continue to expand the capabilities of our working example of a Flyer.  Now, we wish to "collect" a 1-D array of data.  We'll manufacture that data ourselves to keep the code simple.

Here, we generate an array of 5 numbers, each representing the elapsed time (in seconds) since the scan started (in `kickoff()`).  We record the start time with `self.t0 = time.time()`.  Also, `time.time()` is used to generate the timestamps in the data events.  In a real case, we *might* get timestamps from the *controller* or we'd have to make it up ourselves, just like this example.

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/388eb30304e51...a0af3ec57a3430e777b3).

In [18]:
class MyFlyer(ophyd.Device):
    """
    a Flyer that we understand that reports 1-D array of data
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None
        self.t0 = 0

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self.complete_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self.complete_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self.complete_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        kickoff_status = ophyd.DeviceStatus(self)
        self.complete_status = ophyd.DeviceStatus(self)
        self.t0 = time.time()
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self.complete_status is None:
            raise RuntimeError("No collection in progress")

        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "elapsed time, s",
            dtype = "number",
            shape = (1,)
        )
        return {
            self.name: {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self.complete_status = None
        for _ in range(5):
            t = time.time()
            x = t - self.t0 # data is elapsed time since kickoff()
            d = dict(
                time=t,
                data=dict(x=x),
                timestamps=dict(x=t)
            )
            yield d


In [19]:
ifly = MyFlyer(name="ifly")

In [20]:
print(ifly.describe_collect())

{'ifly': {'x': {'source': 'elapsed time, s', 'dtype': 'number', 'shape': (1,)}}}


In [21]:
list(ifly.collect())

[{'data': {'x': 1525113619.1373274},
  'time': 1525113619.1373274,
  'timestamps': {'x': 1525113619.1373274}},
 {'data': {'x': 1525113619.1373296},
  'time': 1525113619.1373296,
  'timestamps': {'x': 1525113619.1373296}},
 {'data': {'x': 1525113619.1373312},
  'time': 1525113619.1373312,
  'timestamps': {'x': 1525113619.1373312}},
 {'data': {'x': 1525113619.1373327},
  'time': 1525113619.1373327,
  'timestamps': {'x': 1525113619.1373327}},
 {'data': {'x': 1525113619.1373343},
  'time': 1525113619.1373343,
  'timestamps': {'x': 1525113619.1373343}}]

Again, not much information from running this flyer, except that it succeeds and a uuid is returned.

In [22]:
RE(bluesky.plans.fly([ifly]))

('2d75bd46-c057-421b-af41-3f7db42124eb',)

More information is obtained by asking the databroker about the most recent scan.  Here's the data table from our stream of data events.  (The stream is named "ifly" as set from `self.name` in the `describe_collect()` method.)  We'll reference that last scan as `h` to reduce the amount of typing.

In [23]:
h = db[-1]
h.stream_names[0]

'ifly'

In [24]:
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2018-04-30 14:40:19.180633,0.014571
2,2018-04-30 14:40:19.180691,0.014629
3,2018-04-30 14:40:19.180706,0.014644
4,2018-04-30 14:40:19.180720,0.014657
5,2018-04-30 14:40:19.180733,0.014671


We generated 5 data points, there are 5 `event` documents to match.

In [25]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 4,
   'time': 1525113619.1612775,
   'uid': '2d75bd46-c057-421b-af41-3f7db42124eb'}),
 ('descriptor',
  {'data_keys': {'x': {'dtype': 'number',
     'shape': (1,),
     'source': 'elapsed time, s'}},
   'hints': {},
   'name': 'ifly',
   'object_keys': {'ifly': ['x']},
   'run_start': '2d75bd46-c057-421b-af41-3f7db42124eb',
   'time': 1525113619.1689386,
   'uid': '755cd8da-b72b-47f8-9f0a-6e6a59bb0a1f'}),
 ('event',
  {'data': {'x': 0.01457071304321289},
   'descriptor': '755cd8da-b72b-47f8-9f0a-6e6a59bb0a1f',
   'filled': {},
   'seq_num': 1,
   'time': 1525113619.180633,
   'timestamps': {'x': 1525113619.180633},
   'uid': 'da947306-d0e3-454f-96ff-704375279940'}),
 ('event',
  {'data': {'x': 0.01462864875793457},
   'descriptor': '755cd8da-b72b-47f8-9f0a-6e6a59bb0a1f',
   'filled': {},
   'seq_num': 2,
   'time': 1525113619.180691,
   'timestamps': {'x': 1525113619.180691},
   'uid': '53fa55b0-7e4b-4a23-9ca8

## Conclusion

Now you have seen the basic steps in creating a `Flyer`.  You have also been shown a few diagnostic tools to help you investigate your code development.